In [82]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import seaborn as sns 
import matplotlib.pyplot as plt 
import plotly.express as px

import warnings
warnings.filterwarnings("ignore")

## API Buenos Aires BA DATA

### Lectura del dataset *Homicidios*

Leemos el archivo CSV y lo convertimo en un DataFrame.

In [121]:
homicidios = pd.read_csv('C:/Users/Raul Penayo/Desktop/p2ori/homicidios_hechos.csv')

In [129]:
homicidios.head()

,Id Hecho,Nro de víctimas,Fecha,Año,Mes,Día,Hora,Hora entera,Lugar del hecho,Tipo de calle,...,Comuna,XY (CABA),POS X,POS Y,Participantes,Víctima,Acusado,Día de la Semana,Tipo de Día,barrio
0,2016-0001,1,2016-01-01,2016,Enero,1,04:00:00,4.0,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,...,8,Point (98896.78238426 93532.43437792),-58.475340,-34.687570,MOTO-AUTO,MOTO,AUTO,Viernes,Día de Semana,Villa Riachuelo
1,2016-0002,1,2016-01-02,2016,Enero,2,01:15:00,1.0,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,...,9,Point (95832.05571093 95505.41641999),-58.508775,-34.669777,AUTO-PASAJEROS,AUTO,PASAJEROS,Sábado,Fin de Semana,Sobre Gral. Paz
2,2016-0003,1,2016-01-03,2016,Enero,3,07:00:00,7.0,AV ENTRE RIOS 2034,AVENIDA,...,1,Point (106684.29090040 99706.57687843),-58.390403,-34.631894,MOTO-AUTO,MOTO,AUTO,Domingo,Fin de Semana,Constitucion
3,2016-0004,1,2016-01-10,2016,Enero,10,00:00:00,0.0,AV LARRAZABAL Y GRAL VILLEGAS CONRADO,AVENIDA,...,8,Point (99840.65224780 94269.16534422),-58.465039,-34.680930,MOTO-SD,MOTO,SD,Domingo,Fin de Semana,Villa Lugano
4,2016-0005,1,2016-01-21,2016,Enero,21,05:20:00,5.0,AV SAN JUAN Y PRESIDENTE LUIS SAENZ PEÑA,AVENIDA,...,1,Point (106980.32827929 100752.16915795),-58.387183,-34.622466,MOTO-PASAJEROS,MOTO,PASAJEROS,Jueves,Día de Semana,Constitucion


In [130]:
import requests

def obtener_barrio(latitud, longitud):
    """
    Obtiene el nombre del barrio correspondiente a las coordenadas geográficas dadas.

    Parameters:
    -----------
        latitud (float): La latitud de la ubicación.
        longitud (float): La longitud de la ubicación.

    Returns:
    --------
        str: El nombre del barrio si se encuentra, o "Desconocido" si no se encuentra o hay un error.
    """
    url = "https://datosabiertos-usig-apis.buenosaires.gob.ar/datos_utiles"
    params = {
        "x": longitud,
        "y": latitud,
        "formato": "json"
    }

    try:
        response = requests.get(url, params=params)
        response.raise_for_status()  # Esto lanzará una excepción para respuestas con código de estado 4xx/5xx
        data = response.json()
        return data.get("barrio", "Desconocido")
    except requests.exceptions.RequestException as e:
        print("Error al hacer la solicitud:", e)
        return "Desconocido"

# Ejemplo de uso
latitud = -34.603722
longitud = -58.381592
barrio = obtener_barrio(latitud, longitud)
print(barrio)


San Nicolas


### Creación de una función

Creamos una función para poder obtener a que Barrio pertenece de **CABA** apartir de los datos que tenemos de *Latitud* y *Longitud* en el DataFrame

In [131]:
import pandas as pd
import requests

# usamos  el DataFrame cargado como homicidios
homicidios = pd.read_csv('C:/Users/Raul Penayo/Desktop/p2ori/homicidios_hechos.csv')  # Carga tu DataFrame desde el archivo adecuado

def obtener_barrio(latitud, longitud):
    """
    Obtiene el nombre del barrio correspondiente a las coordenadas geográficas dadas.

    Parameters:
    -----------
        latitud (float): La latitud de la ubicación.
        longitud (float): La longitud de la ubicación.

    Returns:
    --------
        str: El nombre del barrio si se encuentra, o "Desconocido" si no se encuentra o hay un error.
    """
    url = "https://datosabiertos-usig-apis.buenosaires.gob.ar/datos_utiles"
    params = {
        "x": longitud,
        "y": latitud,
        "formato": "json"
    }

    try:
        response = requests.get(url, params=params)
        response.raise_for_status()  # Esto lanzará una excepción para respuestas con código de estado 4xx/5xx

        # Añadir información de depuración
        print(f"Solicitud exitosa para coordenadas: latitud={latitud}, longitud={longitud}")
        print(f"Contenido de la respuesta: {response.content[:100]}...")  # Muestra los primeros 100 caracteres

        # Comprueba si la respuesta contiene datos antes de decodificar como JSON
        if response.content.strip():  # strip() para eliminar espacios en blanco
            data = response.json()
            return data.get("barrio", "Desconocido")
        else:
            print("Respuesta vacía del servidor para coordenadas:", latitud, longitud)
            return "Desconocido"

    except requests.exceptions.RequestException as e:
        print("Error al hacer la solicitud:", e)
        return "Desconocido"
    except ValueError as e:
        # Captura el error de JSON inválido
        print("Error al decodificar la respuesta JSON:", e, "para coordenadas:", latitud, longitud)
        return "Desconocido"

# Aplica la función obtener_barrio a cada fila del DataFrame homicidios
homicidios['barrio'] = homicidios.apply(lambda row: obtener_barrio(row['POS Y'], row['POS X']), axis=1)

# Muestra el DataFrame con la nueva columna
print(homicidios[['POS X', 'POS Y', 'barrio']])


Solicitud exitosa para coordenadas: latitud=-34.68757022, longitud=-58.47533969
Contenido de la respuesta: b'{"comuna":"Comuna 8","barrio":"Villa Riachuelo","comisaria":"52","area_hospitalaria":"HTAL. DRA. CEC'...
Solicitud exitosa para coordenadas: latitud=-34.66977709, longitud=-58.50877521
Contenido de la respuesta: b'{"comuna":"","barrio":"","comisaria":"","area_hospitalaria":"","region_sanitaria":"","distrito_escol'...
Solicitud exitosa para coordenadas: latitud=-34.63189362, longitud=-58.39040293
Contenido de la respuesta: b'{"comuna":"Comuna 1","barrio":"Constitucion","comisaria":"28","area_hospitalaria":"HTAL. J.M. PENNA"'...
Solicitud exitosa para coordenadas: latitud=-34.68092974, longitud=-58.46503904
Contenido de la respuesta: b'{"comuna":"Comuna 8","barrio":"Villa Lugano","comisaria":"52","area_hospitalaria":"HTAL. DRA. CECILI'...
Solicitud exitosa para coordenadas: latitud=-34.6224663, longitud=-58.38718297
Contenido de la respuesta: b'{"comuna":"Comuna 1","barrio":"Const

Nos cercioramos de que este bien 

In [132]:
homicidios[['barrio']]

,barrio
0,Villa Riachuelo
1,
2,Constitucion
3,Villa Lugano
4,Constitucion
...,...
689,Parque Chacabuco
690,Parque Avellaneda
691,Flores
692,Parque Avellaneda


### Análisis de los datos

Analizamos los datos que obtuvimos de la API, ya que notamos que hay registros sin datos de Barrio.

In [133]:
homicidios[homicidios.barrio == '']

,Id Hecho,Nro de víctimas,Fecha,Año,Mes,Día,Hora,Hora entera,Lugar del hecho,Tipo de calle,...,Comuna,XY (CABA),POS X,POS Y,Participantes,Víctima,Acusado,Día de la Semana,Tipo de Día,barrio
1,2016-0002,1,2016-01-02,2016,Enero,2,01:15:00,1.0,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,...,9,Point (95832.05571093 95505.41641999),-58.508775,-34.669777,AUTO-PASAJEROS,AUTO,PASAJEROS,Sábado,Fin de Semana,
7,2016-0010,1,2016-01-29,2016,Enero,29,15:20:00,15.0,AV GENERAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,...,9,Point (95832.05571093 95505.41641999),-58.508775,-34.669777,MOTO-AUTO,MOTO,AUTO,Viernes,Día de Semana,
17,2016-0025,1,2016-02-28,2016,Febrero,28,04:50:00,4.0,AV GRAL PAZ Y AV 27 DE FEBRERO,GRAL PAZ,...,8,Point (100077.95205948 91571.23307293),-58.462450,-34.705250,AUTO-AUTO,AUTO,AUTO,Domingo,Fin de Semana,
27,2016-0038,1,2016-03-21,2016,Marzo,21,08:06:00,8.0,AV GRAL PAZ COLECTORA Y ZUVIRIA,GRAL PAZ,...,8,Point (96926.79887463 94490.76318635),-58.496835,-34.678927,PEATON-PASAJEROS,PEATON,PASAJEROS,Lunes,Día de Semana,
32,2016-0045,1,2016-04-11,2016,Abril,11,10:25:00,10.0,AV GRAL PAZ Y AV DEL LIBERTADOR,GRAL PAZ,...,13,Point (99620.34936816 110483.29286598),-58.467435,-34.534769,MOTO-SD,MOTO,SD,Lunes,Día de Semana,
40,2016-0054,1,2016-04-23,2016,Abril,23,21:00:00,21.0,AU LUIS DELLEPIANE Y AV GRAL PAZ,AUTOPISTA,...,8,Point (97832.57143731 93489.83736533),-58.486954,-34.687952,MOTO-MOTO,MOTO,MOTO,Sábado,Fin de Semana,
51,2016-0074,1,2016-06-12,2016,Junio,12,06:40:00,6.0,AV GRAL PAZ Y RODOLFO SCAPINO,GRAL PAZ,...,8,Point (96731.44252651 94668.88351657),-58.498966,-34.677321,AUTO-AUTO,AUTO,AUTO,Domingo,Fin de Semana,
68,2016-0092,1,2016-07-15,2016,Julio,15,11:00:00,11.0,AV GRAL PAZ Y AV LIBERTADOR,GRAL PAZ,...,13,Point (99620.34936816 110483.29286598),-58.467435,-34.534769,MOTO-CARGAS,MOTO,CARGAS,Viernes,Día de Semana,
74,2016-0099,1,2016-07-29,2016,Julio,29,16:00:00,16.0,AV GRAL MOSCONI Y AV GRAL PAZ,GRAL PAZ,...,11,Point (94486.28631712 103468.59928582),-58.523406,-34.597988,CARGAS-CARGAS,CARGAS,CARGAS,Viernes,Día de Semana,
129,2016-0163,1,2016-12-10,2016,Diciembre,10,07:30:00,7.0,AV GRAL PAZ Y AV GENERAL MOSCONI,GRAL PAZ,...,11,Point (94486.28631712 103468.59928582),-58.523406,-34.597988,MOTO-AUTO,MOTO,AUTO,Sábado,Fin de Semana,


Observamos que hay muchos registros sin datos de Barrio, veamos cuantos.

In [134]:
homicidios[homicidios.barrio == ''].shape[0]

31

Hay 31 registro sin datos de Barrio, y notamos que el *Tipo de calle* de esos registros son **GRAL PAZ**. <br>
Veamos estos registros en un mapa

In [135]:
homicidios_sin_barrio = homicidios[homicidios['barrio'] == '']

In [136]:
fig = px.scatter_mapbox(
    homicidios_sin_barrio,
    lat = 'POS Y',
    lon = 'POS X',
    zoom = 10.8,
    width = 800,   
    height = 680 
)

fig.update_layout(mapbox_style = 'open-street-map')
fig.update_layout(title_x=0.5,
                  title_y=0.95, 
                  title_text='Siniestros sin datos de Barrio',
                  title_font=dict(size=24))
fig.update_traces(marker=dict(size=7, color='#113946'))  


fig.show()

Obseramos que estos siniestros ocurrieron sobre la Gral. Paz o al límite de la Ciudad Autonoma de Buenos Aires. Asi que procederemos a imputarlo como ***Sobre Gral Paz***  para dar a entender los datos.

In [137]:
homicidios['barrio'] = homicidios['barrio'].replace('', 'Sobre Gral. Paz')

Guardamos el data frame 

In [138]:
homicidios.to_csv('C:/Users/Raul Penayo/Desktop/p2ori/homicidios_hechos.csv', index=False, encoding='utf-8')

## Página de *Wikipedia*

La página que usaremos es esta: https://es.wikipedia.org/wiki/Censo_argentino_de_2022, el cual tiene información estadístico del Censo del 2010 y 2022 categorizado por *Jurisdicción* y *Aglomerado Urbano*.

In [105]:
url = 'https://es.wikipedia.org/wiki/Censo_argentino_de_2022'

# Realizamos una solicitud GET a la página
response = requests.get(url)

# Analizamos el contenido HTML de la página
soup = BeautifulSoup(response.content, 'html.parser')

# Buscamos la segunda tabla de la pagina con la clase 'wikitable'
tabla = soup.find_all('table', class_='wikitable')[0]

# Extraemos los datos de la tabla y lo guardamos en una lista
datos = []
filas = tabla.find_all('tr')

for fila in filas[1:]:  
    celdas = fila.find_all(['th', 'td'])
    fila_datos = [celda.get_text(strip=True) for celda in celdas]
    datos.append(fila_datos)

Vemos los datos y seleccionamos los que corresponden a la ciudad autonoma de buenos aires 

In [106]:
for fila in datos:
    print(fila)

['1', 'Provincia de Buenos Aires', '15\xa0625\xa0084[10]\u200b', '17\xa0523\xa0996[11]\u200b', '12,1', '9,8']
['2', 'Córdoba', '3\xa0308\xa0876', '3\xa0840\xa0905', '16,1', '15,5']
['3', 'Santa Fe', '3\xa0194\xa0537', '3\xa0544\xa0908', '11,0', '9,0']
['4', 'Ciudad Autónoma de Buenos Aires', '2\xa0890\xa0151', '3\xa0121\xa0707', '8,0', '6,4']
['5', 'Mendoza', '1\xa0738\xa0929', '2\xa0043\xa0540', '17,5', '12,3']
['6', 'Tucumán', '1\xa0448\xa0188', '1\xa0731\xa0820', '19,6', '13,6']
['7', 'Salta', '1\xa0214\xa0441', '1\xa0441\xa0351', '18,7', '14,3']
['8', 'Entre Ríos', '1\xa0235\xa0994', '1\xa0425\xa0578', '15,3', '12,0']
['9', 'Misiones', '1\xa0101\xa0593', '1\xa0278\xa0873', '16,1', '12,7']
['10', 'Corrientes', '992\xa0595', '1\xa0212\xa0696', '22,2', '15,8']
['11', 'Chaco', '1\xa0055\xa0259', '1\xa0129\xa0606', '7,0', '6,7']
['12', 'Santiago del Estero', '874\xa0006', '1\xa0060\xa0906', '21,4', '15,7']
['13', 'San Juan', '681\xa0055', '822\xa0853', '20,8', '15,4']
['14', 'Jujuy', '6

In [107]:
data_caba = datos[3]

In [108]:
data_caba

['4',
 'Ciudad Autónoma de Buenos Aires',
 '2\xa0890\xa0151',
 '3\xa0121\xa0707',
 '8,0',
 '6,4']

seleccionamos los valores del 2010 y el 2022 de la lista

In [109]:
poblacion_2010 = int(data_caba[2].replace('.', '').replace(',', ''))
poblacion_2022 = int(data_caba[3].replace('.', '').replace(',', ''))

ValueError: invalid literal for int() with base 10: '2\xa0890\xa0151'

vemos que al intentar convertir el str a un entero notamos que contiene caracteres no numericos,y el metodo replace en este caso esta mal aplicado, asi que procedemos a rearmar el codigo.

In [110]:
poblacion_2010 = int(data_caba[2].replace('.', '').replace(',', '').replace('\xa0', ''))
poblacion_2022 = int(data_caba[3].replace('.', '').replace(',', '').replace('\xa0', ''))


Creamos una lista de población por año calculando el crecimiento anual promedio.

In [111]:
anios = list(range(2010, 2023))

poblacion_por_anio = [poblacion_2010 + (poblacion_2022 - poblacion_2010) * (anio - 2010) / 12 for anio in anios]

poblacion_por_anio = list(map(int, poblacion_por_anio))

In [112]:
poblacion_por_anio

[2890151,
 2909447,
 2928743,
 2948040,
 2967336,
 2986632,
 3005929,
 3025225,
 3044521,
 3063818,
 3083114,
 3102410,
 3121707]

creamos un data frame con los datos obtenidos 

In [113]:
data = {'Año': anios, 'Población': poblacion_por_anio}
poblacion_caba = pd.DataFrame(data)

In [114]:
poblacion_caba

,Año,Población
0,2010,2890151
1,2011,2909447
2,2012,2928743
3,2013,2948040
4,2014,2967336
5,2015,2986632
6,2016,3005929
7,2017,3025225
8,2018,3044521
9,2019,3063818


Guardamos el dataframe un archivo csv

In [116]:
poblacion_caba.to_csv('C:/Users/Raul Penayo/Desktop/p2ori/poblacion_caba.csv', index= False, encoding='utf-8')

In [127]:
homicidios_victimas.head()


,Id Hecho,Fecha,Año,Mes,Día,Rol,Víctima,Sexo,Edad,Fecha de fallecimiento
0,2016-0001,2016-01-01,2016,1,1,CONDUCTOR,MOTO,MASCULINO,19,2016-01-01 00:00:00
1,2016-0002,2016-01-02,2016,1,2,CONDUCTOR,AUTO,MASCULINO,70,2016-01-02 00:00:00
2,2016-0003,2016-01-03,2016,1,3,CONDUCTOR,MOTO,MASCULINO,30,2016-01-03 00:00:00
3,2016-0004,2016-01-10,2016,1,10,CONDUCTOR,MOTO,MASCULINO,18,SD
4,2016-0005,2016-01-21,2016,1,21,CONDUCTOR,MOTO,MASCULINO,29,2016-02-01 00:00:00


In [125]:
homicidios_victimas= pd.read_csv('C:/Users/Raul Penayo/Desktop/p2ori/homicidios_víctimas.csv')